In [1]:
import pandas as pd
import os
import csv

## Cleanup airport hub data

In [ ]:
# File paths
airport_hubs_path = 'airport hubs.csv'
airports_cleaned_path = 'Airports_Cleaned.csv'

# Load the CSV files into DataFrames
airport_hubs_df = pd.read_csv(airport_hubs_path)
airports_cleaned_df = pd.read_csv(airports_cleaned_path)

# Perform the join using 'Airport ID' from airport_hubs.csv and 'ARPT_ID' from Airports_Cleaned.csv
merged_df = airport_hubs_df.merge(airports_cleaned_df, left_on='ARPT_ID', right_on='ARPT_ID', how='inner')

# Drop the specified columns
columns_to_drop = ['COUNTRY_CODE', 'OWNERSHIP_TYPE_CODE', 'FACILITY_USE_CODE', 'STATE_CODE' ]
merged_df = merged_df.drop(columns=columns_to_drop)

merged_df

In [5]:
# Save the output to a new CSV file
merged_df.to_csv('../Final_Cleaned_Data/Airport_Hub_List.csv', index=False)

# Display a message
print("Merged data has been saved")

Merged data has been saved


## Weather cleanup and data engineering processes

In [ ]:
# Update weather csv files to include ARPT_ID column and populate with Airport ID
# Path to the weather data directory
weather_2023_directory = r'D:\_Bootcamp\_Project4\Redo\Weather\W_2021'

# Iterate through each CSV file in the Weather data directory
for file in os.listdir(weather_2023_directory):
    if file.endswith('.csv'):  # Process only CSV files
        file_path = os.path.join(weather_2023_directory, file)
        
        # Extract the first 3 letters of the file name
        arpt_id = file[:3]
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Add the ARPT_ID column
        df['ARPT_ID'] = arpt_id
        
        # Save the updated DataFrame back to the same file
        df.to_csv(file_path, index=False)
        
        print(f"Updated file: {file} with ARPT_ID = {arpt_id}")

# Summary message
print("All CSV files in the Weather_2023 directory have been updated with the ARPT_ID column.")

In [ ]:
# Directories for the CSV files
weather_2019_directory = '\Weather\Weather_2019'
weather_2020_directory = '\Weather\Weather_2020'
weather_2021_directory = '\Weather\Weather_2021'
weather_2022_directory = '\Weather\Weather_2022'
weather_2023_directory = '\Weather\Weather_2023'
weather_2024_directory = '\Weather\Weather_2024'
output_file = '\Weather\Cleaned\Merged_Weather.csv'

# Combine all directories into a list
directories = [weather_2019_directory, weather_2020_directory, weather_2021_directory, weather_2022_directory, weather_2023_directory, weather_2024_directory]

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate through each directory and process CSV files
for directory in directories:
    for file in os.listdir(directory):
        if file.endswith('.csv'):  # Process only CSV files
            file_path = os.path.join(directory, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

# Concatenate all DataFrames
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a single CSV file
merged_df.to_csv(output_file, index=False)

# Display a summary message
print(f"All CSV files from weather data have been merged.")

In [13]:
# Weather data engineering
# Load the dataset
file_path = '\Weather\Cleaned\Merged_Weather.csv'
weather_data = pd.read_csv(file_path)

# Define thresholds for feature engineering
extreme_heat_threshold = 49  # °C
extreme_cold_threshold = -20  # °C
severe_cold_threshold = -40  # °C
strong_wind_speed_threshold_kmh = 46.3  # km/h (~25 knots)
severe_wind_speed_threshold_kmh = 74.1  # km/h (~40 knots)
heavy_precipitation_threshold = 10  # mm
extreme_precipitation_threshold = 50  # mm

# Create binary features for temperature thresholds
weather_data['extreme_heat'] = weather_data['temperature_2m_max'] > extreme_heat_threshold
weather_data['extreme_cold'] = weather_data['temperature_2m_min'] < extreme_cold_threshold
weather_data['severe_cold'] = weather_data['temperature_2m_min'] < severe_cold_threshold

# Create binary features for precipitation thresholds
weather_data['light_precipitation'] = weather_data['precipitation_sum'] < heavy_precipitation_threshold
weather_data['moderate_precipitation'] = (weather_data['precipitation_sum'] >= heavy_precipitation_threshold) & \
                                         (weather_data['precipitation_sum'] < extreme_precipitation_threshold)
weather_data['heavy_precipitation'] = weather_data['precipitation_sum'] >= extreme_precipitation_threshold

# Create binary feature for snowfall with freezing temperatures
weather_data['freezing_precipitation'] = (weather_data['snowfall_sum'] > 0) & \
                                         (weather_data['temperature_2m_min'] <= 0)

# Create binary features for wind speed thresholds (converted to km/h)
weather_data['strong_winds'] = weather_data['wind_speed_10m_max'] * 3.6 > strong_wind_speed_threshold_kmh
weather_data['severe_winds'] = weather_data['wind_gusts_10m_max'] * 3.6 > severe_wind_speed_threshold_kmh

# Composite indicator: High likelihood of delay (any extreme condition met)
weather_data['high_delay_likelihood'] = weather_data[
    ['extreme_heat', 'extreme_cold', 'heavy_precipitation', 'severe_winds']
].any(axis=1)

# Save the updated dataset
updated_file_path = '\Weather\Cleaned\merge_w_data_engineered .csv'
weather_data.to_csv(updated_file_path, index=False)

print(f"Updated weather data with engineered features saved.")

Updated weather data with engineered features saved.


In [9]:
# Display weather data csv data type
weather_df = pd.read_csv('Merged_Weather_Data_Completed.csv') 

# Display data types of each column
print(weather_df.dtypes)

date                            object
latitude                       float64
longitude                      float64
weather_code                   float64
temperature_2m_max             float64
temperature_2m_min             float64
precipitation_sum              float64
snowfall_sum                   float64
precipitation_hours            float64
wind_speed_10m_max             float64
wind_gusts_10m_max             float64
wind_direction_10m_dominant    float64
ARPT_ID                         object
extreme_heat                      bool
extreme_cold                      bool
severe_cold                       bool
light_precipitation               bool
moderate_precipitation            bool
heavy_precipitation               bool
freezing_precipitation            bool
strong_winds                      bool
severe_winds                      bool
high_delay_likelihood             bool
id                             float64
dtype: object


In [24]:
# Read the CSV file
weather_df = pd.read_csv('Merged_Weather_Data_Completed.csv') 

# Change date format 
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.strftime('%m/%d/%Y') 

# Export the modified DataFrame to a new CSV file
weather_df.to_csv('Merged_Weather_Data_date.csv', index=False)

In [20]:
# Convert boolean columns True = 1, False = 0
# Path to the input CSV file
input_file = 'Merged_Weather_Data_Completed.csv'

# Path to the output CSV file
output_file = 'Merged_Weather_Data_Completed_binary.csv'

# Read the CSV file
df = pd.read_csv(input_file)

# Convert True/False to 1/0 for all columns
df = df.replace({True: 1, False: 0})

# Save the updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"Conversion complete. Updated file saved as: {output_file}")


Conversion complete. Updated file saved as: D:\_Bootcamp\_Project4\Data\Merged_Weather_Data_Completed_binary.csv


## Flight delay cleanup process and data engineering processes

In [6]:
# Extract flight delay zipped files to a folder
# Function to extract all zipped files in a folder
def extract_zipped_files(source_folder, target_folder):
    # Create target folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # Loop through files in the source folder
    for file_name in os.listdir(source_folder):
        if file_name.endswith(".zip"):
            file_path = os.path.join(source_folder, file_name)
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                print(f"Extracting {file_name}...")
                zip_ref.extractall(target_folder)
    
    print("Extraction completed.")

# Specify the source folder containing the zip files
source_folder = 'Project4\_Archive"

# Specify the target folder where you want to extract the files
target_folder = 'Flight_delays\part1_csv"

# Run the function
extract_zipped_files(source_folder, target_folder)


Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_1.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_10.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_11.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_12.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_2.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_3.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_4.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_5.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_6.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2022_7.zip...
Extracting On_Time_Marketing_Carrier_On_Time_Performance_

In [7]:
# Remove unneeded columns in the flight delay data

import warnings
warnings.filterwarnings("ignore")

# Path to the Flight_Delays folder
flight_delays_directory = 'Flight_delays\part1_csv'

# Columns to keep
columns_to_keep = [
    'Year', 'FlightDate', 'Operating_Airline ', 'Origin', 
    'OriginStateName', 'Dest', 'DestStateName', 
    'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 
    'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 
    'Cancelled', 'CancellationCode', 'WeatherDelay', 'NASDelay',
    'SecurityDelay', 'CarrierDelay', 'LateAircraftDelay'
]

# Iterate through each CSV file in the Flight_Delays directory
for file in os.listdir(flight_delays_directory):
    if file.endswith('.csv'):  # Process only CSV files
        file_path = os.path.join(flight_delays_directory, file)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Keep only the specified columns, ignoring missing ones
        filtered_df = df.loc[:, df.columns.intersection(columns_to_keep)]
        
        # Save the filtered DataFrame back to the same file
        filtered_df.to_csv(file_path, index=False)
        
        print(f"Processed file: {file} (kept specified columns)")

# Summary message
print("All CSV files in the Flight_Delays directory have been updated to retain only the specified columns.")

Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_1.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_10.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_11.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_12.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_2.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_3.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_4.csv (kept specified columns)
Processed file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_5.csv (kept specified columns)
Processed fil

In [8]:
# Keeping records that matched with target ARPT_ID
# File paths
airport_hub_list_path = 'Data_final\Airport_Hub_List.csv'
flight_delays_directory = 'Flight_delays'
output_directory = 'Flight_delays\csv\output_matched_APRT'

# Load the Airport Hub List
airport_hub_list_df = pd.read_csv(airport_hub_list_path)

# Ensure the ARPT_ID column exists
if 'ARPT_ID' not in airport_hub_list_df.columns:
    raise KeyError("The column 'ARPT_ID' does not exist in Airport_Hub_List.csv.")

# Get the list of ARPT_IDs
arpt_ids = airport_hub_list_df['ARPT_ID'].unique()

# Iterate through each CSV file in the Flight_Delays directory
for file in os.listdir(flight_delays_directory):
    if file.endswith('.csv'):  # Process only CSV files
        file_path = os.path.join(flight_delays_directory, file)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Filter rows where Origin or Dest match any ARPT_ID
        filtered_df = df[(df['Origin'].isin(arpt_ids)) & (df['Dest'].isin(arpt_ids))]
        
        # Save the filtered DataFrame to the output directory
        filtered_file_path = os.path.join(output_directory, file)
        filtered_df.to_csv(filtered_file_path, index=False)
        
        print(f"Filtered file: {file} (saved to {filtered_file_path})")

# Summary message
print(f"All CSV files in the Flight_Delays directory have been filtered and saved.")


Filtered file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_1.csv (saved to D:\_Bootcamp\_Project4\Redo\Flight_delays\csv\output_matched_APRT\On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_1.csv)
Filtered file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_10.csv (saved to D:\_Bootcamp\_Project4\Redo\Flight_delays\csv\output_matched_APRT\On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_10.csv)
Filtered file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_11.csv (saved to D:\_Bootcamp\_Project4\Redo\Flight_delays\csv\output_matched_APRT\On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_11.csv)
Filtered file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_12.csv (saved to D:\_Bootcamp\_Project4\Redo\Flight_delays\csv\output_matched_APRT\On_Time_Marketing_Carrier_On_Time_Performance_(Beginni

In [16]:
""" Update columns CancellationCode -> replaces blank/NaN with "Z". 
    Columns CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay -> replaces blank/NaN with 0 """

# Path to the folder containing the CSV files
folder_path = 'Flight_delays\csv\output_matched_APRT'

# List of delay columns to update
delay_columns = [
    'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'
]

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Process only CSV files
        file_path = os.path.join(folder_path, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Update the 'CancellationCode' column
        if 'CancellationCode' in df.columns:
            df['CancellationCode'].fillna('Z', inplace=True)
            df['CancellationCode'].replace('', 'Z', inplace=True)

        # Update delay columns with 0 for blanks or NaNs
        for column in delay_columns:
            if column in df.columns:
                df[column].fillna(0, inplace=True)
                df[column].replace('', 0, inplace=True)
        
        # Save the updated CSV file
        df.to_csv(file_path, index=False)
        print(f"Updated {file_name}")

print("All CSV files in the folder have been updated.")


Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_1.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_10.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_11.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_12.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_2.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_3.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_4.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_5.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_6.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_7.csv
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_8.csv
Updated

In [18]:
# Data Engineering for flight delay data
import warnings
warnings.filterwarnings("ignore")

# Define the folder path containing the flight delay CSV files
flight_delay_folder = 'Flight_delays\csv\output_matched_APRT"
output_folder = 'Flight_delays\Data_engineered"

# Season mapping
season_mapping = {'Winter': 1, 'Spring': 2, 'Summer': 3, 'Fall': 4}

# Iterate over each file in the folder
for filename in os.listdir(flight_delay_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(flight_delay_folder, filename)

        # Read the CSV file
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue

        # Ensure required columns are present
        required_columns = [
            "DepDelayMinutes", "ArrDelayMinutes", "CancellationCode", 
            "CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", 
            "LateAircraftDelay", "Origin", "Dest", "Operating_Airline ", "FlightDate"
        ]
        if all(col in df.columns for col in required_columns):

            # Create the 'Weather_Delayed_Departure' column
            df["Weather_Delayed_Departure"] = df.apply(
                lambda x: 1 if x["DepDelayMinutes"] > 15 and x["WeatherDelay"] > 0 else 0, axis=1
            )

            # Create the 'Weather_Delayed_Arrival' column
            df["Weather_Delayed_Arrival"] = df.apply(
                lambda x: 1 if x["ArrDelayMinutes"] > 15 and x["WeatherDelay"] > 0 else 0, axis=1
            )

            # Create the 'Delayed_Departure' column
            df["Delayed_Departure"] = df["DepDelayMinutes"].apply(lambda x: 1 if x > 15 else 0)

            # Create the 'Delayed_Arrival' column
            df["Delayed_Arrival"] = df["ArrDelayMinutes"].apply(lambda x: 1 if x > 15 else 0)

            # Create the 'Weather_Cancellation' column
            df["Weather_Cancellation"] = df["CancellationCode"].apply(lambda x: 1 if x == "B" else 0)

            # Create the 'Total_Delayed' column
            df["Total_Delayed"] = df[["CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"]].applymap(lambda x: 1 if x > 0 else 0).sum(axis=1)

            # Create the 'Total_Cancellation' column
            df["Total_Cancellation"] = df["CancellationCode"].apply(lambda x: 1 if x in ["A", "B", "C", "D"] else 0)

            # Add 'Month' column
            df["Month"] = pd.to_datetime(df["FlightDate"], errors='coerce').dt.month

            # Add 'Season' column
            df["Season"] = df["Month"].apply(lambda x: 'Winter' if x in [12, 1, 2] else \
                                               'Spring' if x in [3, 4, 5] else \
                                               'Summer' if x in [6, 7, 8] else 'Fall')
            df["Season"] = df["Season"].map(season_mapping)

            # Save the updated DataFrame to the output folder
            output_file_path = os.path.join(output_folder, filename)
            try:
                df.to_csv(output_file_path, index=False)
                print(f"Updated {filename} successfully and saved to output folder.")
            except Exception as e:
                print(f"Error saving {filename}: {e}")
        else:
            print(f"Skipping {filename} - required columns not found.")


Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_1.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_10.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_11.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_12.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_2.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_3.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_4.csv successfully and saved to output folder.
Updated On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2

In [29]:
# Merge Weather data and flight delay data based on Origin Airports
# Define the paths
flight_delay_folder = 'Flight_delays\Data_engineered\archive"
weather_data_path = 'Merged_Weather_Data_Completed.csv"
output_folder = 'Flight_delays\Data_engineered\Origin_merge"

# Load the Weather Data.csv
weather_data = pd.read_csv(weather_data_path)

# Convert date to consistent format
weather_data['date'] = pd.to_datetime(weather_data['date'])

# Process each file in the flight delay folder
for file in os.listdir(flight_delay_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(flight_delay_folder, file)

        # Load flight delay data
        flight_data = pd.read_csv(file_path)
        print(f"Processing file: {file}")

        # Convert FlightDate to datetime
        flight_data['FlightDate'] = pd.to_datetime(flight_data['FlightDate'])

        # Merge with weather data
        merged_data = pd.merge(flight_data, weather_data,
                               left_on=['FlightDate', 'Origin'],
                               right_on=['date', 'ARPT_ID'],
                               how='inner')


        # Drop unnecessary columns
        merged_data_cleaned = merged_data.drop(columns=[
            'latitude', 'longitude', 'weather_code', 'temperature_2m_max', 'temperature_2m_min',
            'precipitation_sum', 'snowfall_sum', 'precipitation_hours', 'wind_speed_10m_max',
            'wind_gusts_10m_max', 'wind_direction_10m_dominant', 'Quarter', 'Flight_Number_Operating_Airline',
            'OriginCityName', 'DestCityName', 'CRSDepTime', 'DepTime',
            'DepDelay', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes',
            'Cancelled', 'CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
            'LateAircraftDelay'
        ], errors='ignore')

        # Create output file name
        output_file_name = f"Updated{file[-11:]}"
        output_file_path = os.path.join(output_folder, output_file_name)

        # Save the merged and cleaned data
        merged_data_cleaned.to_csv(output_file_path, index=False)

print("Merging and cleaning completed. Updated files are saved in the output folder.")

Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_1.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_10.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_11.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_12.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_2.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_3.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_4.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_5.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_6.csv
Processing file: On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2019_7.csv
Process

In [33]:
# Rename columns to add "Origin_" to differentiate weather from origin airports
# Define the paths
flight_delay_folder = 'Flight_delays\Data_engineered\Origin_merge"
output_folder = 'Flight_delays\Data_engineered\Origin_merge\Origin_renamed"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each file in the flight delay folder
for file in os.listdir(flight_delay_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(flight_delay_folder, file)

        # Load flight delay data
        try:
            flight_data = pd.read_csv(file_path)
        except Exception as e:
            print(f"Error loading {file}: {e}")
            continue

        # Drop unnecessary columns
        try:
            flight_data_cleaned = flight_data.drop(columns=['date', 'ARPT_ID', 'high_delay_likelihood'], errors='ignore')
        except Exception as e:
            print(f"Error dropping columns in {file}: {e}")
            continue

        # Rename columns
        rename_columns = {
            'extreme_heat': 'Origin_Extreme Heat',
            'extreme_cold': 'Origin_Extreme Cold',
            'severe_cold': 'Origin_Severe Cold',
            'light_precipitation': 'Origin_Light Precipitation',
            'moderate_precipitation': 'Origin_Moderate Precipitation',
            'heavy_precipitation': 'Origin_Heavy Precipitation',
            'freezing_precipitation': 'Origin_Freezing Precipitation',
            'strong_winds': 'Origin_Strong Winds',
            'severe_winds': 'Origin_Severe Winds'
        }
        try:
            flight_data_cleaned = flight_data_cleaned.rename(columns=rename_columns)
        except Exception as e:
            print(f"Error renaming columns in {file}: {e}")
            continue

        # Create output file name with prefix 'Updated_' and last 7 characters of the original file name
        output_file_name = f"Updated{file[-11:]}"
        output_file_path = os.path.join(output_folder, output_file_name)

        # Save the cleaned data to a new CSV file
        try:
            flight_data_cleaned.to_csv(output_file_path, index=False)
        except Exception as e:
            print(f"Error saving {output_file_name}: {e}")

print("Cleaning and saving completed. Updated files are saved in the output folder.")

Cleaning and saving completed. Updated files are saved in the output folder.


In [32]:
# Drop unneeded columns to reduce file size
# Define the paths
flight_delay_folder = 'Flight_delays\Data_engineered\Origin"
weather_data_path = 'Data_final\Merged_Weather_Data_Completed.csv"
output_folder = 'Flight_delays\Data_engineered\Dest_merged"

# Load the Weather Data.csv
weather_data = pd.read_csv(weather_data_path)

# Convert date to consistent format
weather_data['date'] = pd.to_datetime(weather_data['date'])

# Process each file in the flight delay folder
for file in os.listdir(flight_delay_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(flight_delay_folder, file)

        # Load flight delay data
        flight_data = pd.read_csv(file_path)
        print(f"Processing file: {file}")

        # Convert FlightDate to datetime
        flight_data['FlightDate'] = pd.to_datetime(flight_data['FlightDate'])

        # Merge with weather data
        merged_data = pd.merge(flight_data, weather_data,
                               left_on=['FlightDate', 'Dest'],
                               right_on=['date', 'ARPT_ID'],
                               how='inner')


        # Drop unnecessary columns
        merged_data_cleaned = merged_data.drop(columns=[
            'latitude', 'longitude', 'weather_code', 'temperature_2m_max', 'temperature_2m_min',
            'precipitation_sum', 'snowfall_sum', 'precipitation_hours', 'wind_speed_10m_max',
            'wind_gusts_10m_max', 'wind_direction_10m_dominant', 'Quarter', 'Flight_Number_Operating_Airline',
            'OriginCityName', 'DestCityName', 'CRSDepTime', 'DepTime',
            'DepDelay', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes',
            'Cancelled', 'CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
            'LateAircraftDelay'
        ], errors='ignore')

        # Create output file name
        output_file_name = f"Updated{file[-11:]}"
        output_file_path = os.path.join(output_folder, output_file_name)

        # Save the merged and cleaned data
        merged_data_cleaned.to_csv(output_file_path, index=False)

print("Merging and cleaning completed. Updated files are saved in the output folder.")

Processing file: Updated_2019_1.csv
Processing file: Updated_2019_10.csv
Processing file: Updated_2019_11.csv
Processing file: Updated_2019_12.csv
Processing file: Updated_2019_2.csv
Processing file: Updated_2019_3.csv
Processing file: Updated_2019_4.csv
Processing file: Updated_2019_5.csv
Processing file: Updated_2019_6.csv
Processing file: Updated_2019_7.csv
Processing file: Updated_2019_8.csv
Processing file: Updated_2019_9.csv
Processing file: Updated_2020_1.csv
Processing file: Updated_2020_10.csv
Processing file: Updated_2020_11.csv
Processing file: Updated_2020_12.csv
Processing file: Updated_2020_2.csv
Processing file: Updated_2020_3.csv
Processing file: Updated_2020_4.csv
Processing file: Updated_2020_5.csv
Processing file: Updated_2020_6.csv
Processing file: Updated_2020_7.csv
Processing file: Updated_2020_8.csv
Processing file: Updated_2020_9.csv
Processing file: Updated_2021_1.csv
Processing file: Updated_2021_10.csv
Processing file: Updated_2021_11.csv
Processing file: Upd

In [ ]:
# Clean unneeded columns resulting from join
# Define the paths
flight_delay_folder = 'Flight_delays\Data_engineered\Dest_merged"
output_folder = 'Flight_delays\Data_engineered\Flight_delay_Weather_Data"

# Process each file in the flight delay folder
for file in os.listdir(flight_delay_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(flight_delay_folder, file)

        # Load flight delay data
        try:
            flight_data = pd.read_csv(file_path)
        except Exception as e:
            print(f"Error loading {file}: {e}")
            continue

        # Drop unnecessary columns
        flight_data_cleaned = flight_data.drop(columns=['date', 'ARPT_ID'], errors='ignore')

        # Rename columns
        rename_columns = {
            'extreme_heat': 'Dest_Extreme Heat',
            'extreme_cold': 'Dest_Extreme Cold',
            'severe_cold': 'Dest_Severe Cold',
            'light_precipitation': 'Dest_Light Precipitation',
            'moderate_precipitation': 'Dest_Moderate Precipitation',
            'heavy_precipitation': 'Dest_Heavy Precipitation',
            'freezing_precipitation': 'Dest_Freezing Precipitation',
            'strong_winds': 'Dest_Strong Winds',
            'severe_winds': 'Dest_Severe Winds'
        }
        flight_data_cleaned = flight_data_cleaned.rename(columns=rename_columns)

        # Drop additional columns if present
        flight_data_cleaned = flight_data_cleaned.drop(columns=['high_delay_likelihood'], errors='ignore')

        # Create output file name with prefix 'Updated_'
        output_file_name = f"{file}"
        output_file_path = os.path.join(output_folder, output_file_name)

        # Save the cleaned data to a new CSV file
        try:
            flight_data_cleaned.to_csv(output_file_path, index=False)
            print(f"Saved: {output_file_name}")
        except Exception as e:
            print(f"Error saving {output_file_name}: {e}")

print("Cleaning and saving completed. Updated files are saved in the output folder.")

## Load data to Postgres database

In [1]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# PostgreSQL connection details
host = "localhost"        
port = "5432"              
database = "Project_4_Data"  
user = "postgres"         
password = "postgres" 

# Directory containing your CSV files
csv_directory = 'Flight_delay_Weather_Data"

# Create a connection to PostgreSQL using SQLAlchemy
try:
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")
    print("Connected to PostgreSQL database successfully!")
except Exception as e:
    print(f"Failed to connect to database: {e}")

# Iterate through each CSV file in the directory
for file in os.listdir(csv_directory):
    if file.endswith(".csv"):
        file_path = os.path.join(csv_directory, file)
        
        # Extract table name from the CSV file name
        table_name = os.path.splitext(file)[0]
        
        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(file_path)
        
        # Load DataFrame to PostgreSQL
        try:
            df.to_sql(table_name, engine, if_exists="replace", index=False)
            print(f"Successfully loaded {file} into the {table_name} table.")
        except Exception as e:
            print(f"Error loading {file}: {e}")

# Close the connection
engine.dispose()
print("Data loading complete!")


Connected to PostgreSQL database successfully!
Successfully loaded Updated2019_10.csv into the Updated2019_10 table.
Successfully loaded Updated2019_11.csv into the Updated2019_11 table.
Successfully loaded Updated2019_12.csv into the Updated2019_12 table.
Successfully loaded Updated2020_10.csv into the Updated2020_10 table.
Successfully loaded Updated2020_11.csv into the Updated2020_11 table.
Successfully loaded Updated2020_12.csv into the Updated2020_12 table.
Successfully loaded Updated2021_10.csv into the Updated2021_10 table.
Successfully loaded Updated2021_11.csv into the Updated2021_11 table.
Successfully loaded Updated2021_12.csv into the Updated2021_12 table.
Successfully loaded Updated2022_10.csv into the Updated2022_10 table.
Successfully loaded Updated2022_11.csv into the Updated2022_11 table.
Successfully loaded Updated2022_12.csv into the Updated2022_12 table.
Successfully loaded Updated2023_10.csv into the Updated2023_10 table.
Successfully loaded Updated2023_11.csv into

## Extract data table from Postgres database

In [2]:
import psycopg2

# Database connection details
host = "localhost"
database = "Project_4_Data"
user = "postgres"
password = "postgres"
table_name = "airport_hubs"  # This was reused to export other tables
output_file = r"D:\_Bootcamp\_Project4\Git\Git2\sql_output.csv"

try:
    # Establish the connection
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    print("Connected to the database.")

    # Create a SQL query to select all data from the table
    query = f"SELECT * FROM {table_name};"

    # Use pandas to execute the query and load the data into a DataFrame
    df = pd.read_sql(query, conn)

    # Export the DataFrame to a CSV file
    df.to_csv(output_file, index=False)
    print(f"Table '{table_name}' exported to '{output_file}'.")

except Exception as e:
    print(f"Error: {e}")

finally:
    if conn:
        conn.close()
        print("Database connection closed.")


Connected to the database.
Table 'airport_hubs' exported to 'D:\_Bootcamp\_Project4\Git\Git2\sql_output.csv'.
Database connection closed.


C:\Users\penguin\AppData\Local\Temp\ipykernel_15164\4165947174.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
